In [1]:
# import libraries
import numpy as np
import pandas as pd
import tensorflow as tf
from keras.models import load_model
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
# get data
data = pd.read_csv('local_guide.csv')

# create object TfidfVectorizer
vectorizer = TfidfVectorizer()

# Melakukan vektorisasi TF-IDF pada fitur "Tempat"
tfidf_matrix_tempat = vectorizer.fit_transform(data['Tempat'])

# Mengubah matriks TF-IDF menjadi array
tfidf_matrix = tfidf_matrix_tempat.toarray()

# loads model
model = load_model('model_local_guide.h5')

In [3]:
# Merekomendasikan item berdasarkan Tempat
def recommend_items(tempat, tfidf_matrix, model, items=data[['Pemandu_ID', 'Nama_Pemandu', 'Optional_Bahasa', 'Umur', 'Jenis_Kelamin', 'Tempat', 'Pendidikan_Terakhir', 'Pekerjaan', 'Nomor_Telepon', 'Price_per_hour', 'Time_duration_in_min', 'Avatars', 'Rating']], k=5):
    """This is function to recommend local tour guide

    Args:
        tempat (String): _description_
        tfidf_matrix (_type_): City ​​name
        model (h5): Get model (Sequential, Functional Model, or Model subclass)
        items (params): ndarray, features of the data.
        k (int, optional): recommended number of items, defaults to 5.

    Returns:
        DataFrame: The two-dimensional data structure also contains labeled axes (rows and columns) that contain the recommendation results.
    """
    # Mendapatkan indeks item berdasarkan input Tempat
    def get_item_index_by_tempat(tempat, data):
        index = data[data['Tempat'] == tempat].index
        return index[0] if len(index) > 0 else None

    indeks_item = get_item_index_by_tempat(tempat, data)
    if indeks_item is None:
        return pd.DataFrame()  # Mengembalikan dataframe kosong jika Tempat tidak ditemukan

    # embedding items
    item_embedding = tfidf_matrix[indeks_item]
    item_embedding = np.reshape(item_embedding, (1, -1))

    similarity_scores = cosine_similarity(tfidf_matrix, item_embedding)
    similarity_scores = similarity_scores.flatten()

    indeks_terurut = np.argsort(similarity_scores)[::-1][:k]
    item_terrekomendasikan = items.iloc[indeks_terurut].copy()  # Add .copy() here

    # Mengurutkan berdasarkan Tempat terbaik
    item_terrekomendasikan = item_terrekomendasikan.sort_values('Tempat', ascending=False)

    # Prediksi menggunakan model
    item_embedding_pred = model.predict(item_embedding)
    similarity_scores_pred = cosine_similarity(tfidf_matrix, item_embedding_pred)
    similarity_scores_pred = similarity_scores_pred.flatten()

    indeks_terurut_pred = np.argsort(similarity_scores_pred)[::-1][:k]
    item_terrekomendasikan_pred = items.iloc[indeks_terurut_pred].copy()  # Add .copy() here

    # Mengurutkan berdasarkan Tempat terbaik
    item_terrekomendasikan_pred = item_terrekomendasikan_pred.sort_values('Tempat', ascending=False)

    return item_terrekomendasikan_pred

In [4]:
# run program
tempat_input = 'Bali'  # Ganti dengan input Tempat yang diinginkan

item_terrekomendasikan = recommend_items(tempat_input, tfidf_matrix, model)

1/1 [==============================] - 0s 61ms/step


In [5]:
# show result
if item_terrekomendasikan.empty:
    print('Tidak ada item yang ditemukan untuk Tempat yang diberikan.')
else:
    # Menampilkan item yang direkomendasikan
    print(item_terrekomendasikan[['Pemandu_ID', 'Nama_Pemandu', 'Optional_Bahasa', 'Umur', 'Jenis_Kelamin', 'Tempat', 'Pendidikan_Terakhir', 'Pekerjaan', 'Nomor_Telepon', 'Price_per_hour', 'Time_duration_in_min', 'Avatars', 'Rating']])

     Pemandu_ID                Nama_Pemandu Optional_Bahasa  Umur  \
401     PMD1273               Dadap Nababan         Inggris    19   
1130    PMD1936                Jais Mustofa        Mandarin    30   
1131    PMD1937  H. Muhammad Prabowo, S.E.I         Inggris    29   
834     PMD1667               Lantar Sirait        Mandarin    40   
1538     PMD538             Umaya Sihombing          Jepang    20   

     Jenis_Kelamin Tempat Pendidikan_Terakhir  Pekerjaan  Nomor_Telepon  \
401      Laki-laki   Bali                 SMA        PNS  6285615115173   
1130     Laki-laki   Bali                 SMA  Pengusaha  6288544167597   
1131     Laki-laki   Bali                  S1  Pengusaha  6286929231916   
834      Laki-laki   Bali                 SMP       Guru  6284830979670   
1538     Laki-laki   Bali                 SMP   Wartawan  6281334498718   

      Price_per_hour  Time_duration_in_min  \
401            88000                   180   
1130          108000                   180

In [6]:
# show as dataframe
item_terrekomendasikan

,Pemandu_ID,Nama_Pemandu,Optional_Bahasa,Umur,Jenis_Kelamin,Tempat,Pendidikan_Terakhir,Pekerjaan,Nomor_Telepon,Price_per_hour,Time_duration_in_min,Avatars,Rating
401,PMD1273,Dadap Nababan,Inggris,19,Laki-laki,Bali,SMA,PNS,6285615115173,88000,180,xsgames.co/randomusers/avatar.php?g=male,3
1130,PMD1936,Jais Mustofa,Mandarin,30,Laki-laki,Bali,SMA,Pengusaha,6288544167597,108000,180,xsgames.co/randomusers/avatar.php?g=male,1
1131,PMD1937,"H. Muhammad Prabowo, S.E.I",Inggris,29,Laki-laki,Bali,S1,Pengusaha,6286929231916,137000,60,xsgames.co/randomusers/avatar.php?g=male,1
834,PMD1667,Lantar Sirait,Mandarin,40,Laki-laki,Bali,SMP,Guru,6284830979670,122000,300,xsgames.co/randomusers/avatar.php?g=male,1
1538,PMD538,Umaya Sihombing,Jepang,20,Laki-laki,Bali,SMP,Wartawan,6281334498718,75000,120,xsgames.co/randomusers/avatar.php?g=male,4
